In [ ]:
##Proyecto Final IA  Detector de imágnes de Frutas

#CÓDIGO PARA EL ENTRENAMIENTO
import dataset #forma de importar el set de imágenes
import tensorflow as tf #Forma estandar de importar tensorflow
import time   #Librería que permite trabajar con tiempos de ejecución 
from datetime import timedelta #importar la hora del sistema
import math #librería que permite trabajar con operaciones matemáticas
import random #Librería que permite tra
import numpy as np

#Agregando los datos aleatorio de inicialización
from numpy.random import seed #Trabaja con los números aleatorio.
seed(1) 
from tensorflow import set_random_seed 
set_random_seed(2) #set de numeros aleatorios


batch_size = 32 #Tamaño 32 del lote  es el tamaño mínimo de bits pero que posteriormente se deja a un tamaño estandar de 128

#Prepara los datos de entrada 
classes = ['manzanas','bananos']  #Las clases que se trabajaron clase de manzanas y de bananos
num_classes = len(classes)

# 20% de los datos son usados para la validación 
validation_size = 0.2
img_size = 128          #tamaño standar de las imágenes de 128 bits
num_channels = 3 
train_path='training_data'     #La dirección de la carpeta que contiene las imágenes de entrenamiento.


data = dataset.read_train_sets(train_path, img_size, classes, validation_size=validation_size) #Le y valida las imágenes del entrenamiento


print("Complete reading input data. Will Now print a snippet of it") #Despliegue Datos de entrada
print("Number of files in Training-set:\t\t{}".format(len(data.train.labels))) #Número de archivos en los Datos del entrenamiento
print("Number of files in Validation-set:\t{}".format(len(data.valid.labels))) #Los datos de validación



session = tf.Session() #Se inicia sesión para pasar todos los datos a las variables.
x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_channels], name='x') #Se pasa a la salida una variable por defecto
                     #tipo  forma     fdefault                el canal donde se encuentran los datos    se pasan a x
#Etiquetas
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')#y_true recibe los datos pasados a la variable placeholder
y_true_cls = tf.argmax(y_true, dimension=1)                                  # placeholder recibe el tipo de dato, la forma y el nombre que debe ser verdadero
#y_true_cls recibe el valor máximo y limpia los demas.


##Parámetros de los grafos de la red creada
filter_size_conv1 = 3  #filtra el número de elementos del tensor del grafo 1 en las tres capas 
num_filters_conv1 = 32 #

filter_size_conv2 = 3
num_filters_conv2 = 32

filter_size_conv3 = 3
num_filters_conv3 = 64
    
fc_layer_size = 128  #Capa que contiene el numero de elementos 128 que es el estandar de bits manejado de las imágenes.

def create_weights(shape): #crea la forma de los pesos
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05)) #retorna la variable que contiene la forma y la desvianción standar del tensor al 0.05
                                                                #trunca los otros valores mayores a los considerados en la desviación.
def create_biases(size): #Definimos una funcion con los sesgos
    return tf.Variable(tf.constant(0.05, shape=[size])) #Retorna una variable con los datos constantes de la desviación y forma de los elementos del tensor.



def create_convolutional_layer(input,  #Se define o crea la capa de la red convulucional con las entradas, 
                               
               num_input_channels, #numero de entrada de los canales
               conv_filter_size, #pasa el numero de elementos de los tensores       
               num_filters):#  los filtros o datos obtenidos
    
    ## Determina los pesos usando la función de entranamiento create_weights.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    #Los pesos = se crean los pesos es decir forma, tamaño de los filtros, el canal con los datos de entrada

    biases = create_biases(num_filters)
    #Los filtros que contienen los sesgos.

    # Crea la capa convolucional 
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')

    layer += biases

    ## We shall be using max-pooling.  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    ## Función relu en la capa de salida.
    layer = tf.nn.relu(layer)

    return layer #Retorna la capa

    

def create_flatten_layer(layer):
    #We know that the shape of the layer will be [batch_size img_size img_size num_channels] 
    # But let's get it from the previous layer.
    layer_shape = layer.get_shape()

    ## Number of features will be img_height * img_width* num_channels. But we shall calculate it in place of hard-coding it.
    num_features = layer_shape[1:4].num_elements()

    ## Now, we Flatten the layer so we shall have to reshape to num_features
    layer = tf.reshape(layer, [-1, num_features])

    return layer


def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    # Determina los pesos y los sesgos
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)
    

    # La capa que conecta las x  y produce w*x+b.Since, matmul permite tabajar y operar con la multiplicacion de matrices
    layer = tf.matmul(input, weights) + biases
    if use_relu: #Si la Función de activación use_relu
        layer = tf.nn.relu(layer)

    return layer #Retorna la capa


#recibe los datos de los grafos 

layer_conv1 = create_convolutional_layer(input=x, #grafo 0 o grafo 1 recibe los datos de entrada o x
               num_input_channels=num_channels, #el numero de canal de la cap
               conv_filter_size=filter_size_conv1, #Los números de elementos del tensor
               num_filters=num_filters_conv1) #los filtros calculadors
layer_conv2 = create_convolutional_layer(input=layer_conv1, #grafo 1
               num_input_channels=num_filters_conv1,
               conv_filter_size=filter_size_conv2,
               num_filters=num_filters_conv2)

layer_conv3= create_convolutional_layer(input=layer_conv2, #grafo 2
               num_input_channels=num_filters_conv2,
               conv_filter_size=filter_size_conv3,
               num_filters=num_filters_conv3)
          
layer_flat = create_flatten_layer(layer_conv3)#capa que recibe los datos enlazados entre ellos

layer_fc1 = create_fc_layer(input=layer_flat, 
                     num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs=fc_layer_size,
                     use_relu=True)

layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=fc_layer_size,
                     num_outputs=num_classes,
                     use_relu=False) 

y_pred = tf.nn.softmax(layer_fc2,name='y_pred') #se ejecuta la función de activación softmax a la capa layer_fc2

y_pred_cls = tf.argmax(y_pred, dimension=1) #Se predice el argumento máximo en la dimeónsin 1 y se limpia los demás datos
session.run(tf.global_variables_initializer())#Se corre la sesión para iniciar todas las variables pasadas a la red
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2, # Función de minimización de errores Cross_entropy
                                                    labels=y_true)
cost = tf.reduce_mean(cross_entropy) #costo que es calculado por la media de la función de error
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost) #Se obtimiza la fase de entrenamiento
correct_prediction = tf.equal(y_pred_cls, y_true_cls) #Se determina si nuestra predicción es correcta
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #Se determina la exactitud mediante el calculo de la media de las predicciones


session.run(tf.global_variables_initializer()) #Se core la sesión para iniciar todas las variables


def show_progress(epoch, feed_dict_train, feed_dict_validate, val_loss): #Define el progreso de exactitud despues de haber corrido la seión
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Training Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%},  Validation Loss: {3:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss))

total_iterations = 0 #se inicializa las iteraciones a 0

saver = tf.train.Saver()   
def train(num_iteration): #Se define la funcion del entrenamientos con el numero de iteraciones
    global total_iterations 
    
    for i in range(total_iterations, #Se define un ciclo con el rango de iteraciones
                   total_iterations + num_iteration): #Se incrementan las iteraciones

        x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size) # lectura de paquete o lote de entrenamiento
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(batch_size) #Validacion de los lotes

        
        feed_dict_tr = {x: x_batch,                             
                           y_true: y_true_batch}
        feed_dict_val = {x: x_valid_batch,                       #Se validan lo lotes 
                              y_true: y_valid_batch}

        session.run(optimizer, feed_dict=feed_dict_tr) #Se optimizan los resultados

        if i % int(data.train.num_examples/batch_size) == 0: 
            val_loss = session.run(cost, feed_dict=feed_dict_val) #Se pasa a los valores los datos obtenidos
            epoch = int(i / int(data.train.num_examples/batch_size)) #Se divide los datos de entrenaminto entre el tamaño del lote de entrenamiento
                                                                     # posterior a eso se divien las iteraciones entre los datos del entrenamiento           
            show_progress(epoch, feed_dict_tr, feed_dict_val, val_loss) #Progreso de la fase de predicción
            saver.save(session, 'modelo') #Se guardan los datos obtenidos de la sesión


    total_iterations += num_iteration # Determina que el numero de iteraciones incrementa hasta el total asignado en la variable num_iteration

train(num_iteration=3000) #el entrenamiento recibe como parámetro  3000 iteraciones

Going to read training images
Now going to read manzanas files (Index: 0)
Now going to read bananos files (Index: 1)
Complete reading input data. Will Now print a snippet of it
Number of files in Training-set:		1330
Number of files in Validation-set:	332
Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Use the `axis` argument instead
Training Epoch 1 --- Training Accuracy:  56.2%, Validation Accuracy:  50.0%,  Validation Loss: 0.690
Training Epoch 2 --- Training Accuracy:  78.1%, Validation Accuracy:  62.5%,  Validation Loss: 0.611
Training Epoch 3 --- Training Accuracy:  81.2%, Validation Accuracy:  65.6%,  Validation Loss: 0.637
Training Epoch 4 --- Training Accuracy:  84.4%, Validation Accuracy:  59.4%,  Validation Loss: 0.668
Training Epoch 5 --- Training Accuracy:  84.4%, Validation Accuracy:  71.9%,  Validation Loss: 0.521
Training Epoch 6 --- Training Accuracy:  84.4%, Validation Accuracy:  84.4%,  Validation Loss: 0.428
Training Epoch 7 --- 